<a href="https://colab.research.google.com/github/nemoware/analyser/blob/protocols-4/notebooks/dev_tool_picke_doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pickle
- processing all files and saving results to scv

In [0]:
#@title Config
_git_branch = "protocols-4" #@param {type:"string"}
GLOBALS__={}

In [0]:
''' AZ:-IMPORT CODE GITHUB----------------------------------------------😺------ '''


import tensorflow as tf
import tensorflow_hub as hub

import sys


def _init_import_code_from_gh():
  if 'GLOBALS__' not in globals():
    print('adding global GLOBALS__')
    global GLOBALS__
    GLOBALS__ = {}

  if '_init_import_code_from_gh' in GLOBALS__:
    print('👌 code already imported from GitHub!')
    return

  import subprocess
  def exec(x):
    r = subprocess.check_output(x, shell=True)
    r = r.decode('unicode-escape').encode('latin1').decode('utf8')
    print(r)

  print(f"fetching code from GitHub.....{_git_branch}")
  try:
    exec('rm -r nlp_tools')
  except:
    pass
  exec(f'git clone --single-branch --branch {_git_branch} https://github.com/nemoware/analyser.git nlp_tools')

  print('🦊 GIT revision:')
  exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

  sys.path.insert(0, 'nlp_tools')

  GLOBALS__['_init_import_code_from_gh'] = True

  print('❤️ DONE importing Code fro GitHub')

 

# AZ:-INIT EMBEDDER-----------------------------------------------------------------------------------
def _init_embedder():
  if 'elmo_embedder' in GLOBALS__:
    print('👌 Embedder is already created! ')
    return

  from embedding_tools import ElmoEmbedder
  GLOBALS__['elmo_embedder'] = ElmoEmbedder(module_url = 'https://storage.googleapis.com/az-nlp/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz')
  print('❤️ DONE creating words embedding model')
  return GLOBALS__['elmo_embedder']


def _init_contracts():
  if 'ContractAnlysingContext' in GLOBALS__:
    print('👌 Contracts-related tools are already inited ')
    return

  from contract_parser import ContractAnlysingContext
  GLOBALS__['ContractAnlysingContext'] = ContractAnlysingContext(GLOBALS__['elmo_embedder'], GLOBALS__['renderer'])
  print('❤️ DONE initing Contracts-related tools and models ')


# AZ:- THE CODE----------------------------------------------------------------------------------
def _init_the_code():
  if '_init_the_code' in GLOBALS__:
    print('👌 Code is alredy imported!')
    return

  from renderer import SilentRenderer

  class RendererForBatch(SilentRenderer):
    pass

  GLOBALS__['renderer'] = RendererForBatch()
  GLOBALS__['_init_the_code'] = True


  print("❤️ DONE initializing the code")

# AZ:---------------------------------------------------------------------------END OF THE THE CODE, See you later
import nltk
nltk.download('punkt')



## do preparation here   
    
#1.
_init_import_code_from_gh()
#2.
_init_embedder()
#3.
_init_the_code()
#4. 
# if batch_charters:
# #   _init_charters()
# if batch_contracts:
#   _init_contracts()

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [0]:
!pip install overrides
!pip install pyjarowinkler
from overrides import overrides
# from integration.doc_providers import GDriveTestDocProvider


# doc_provider = GDriveTestDocProvider()

from contract_parser import ContractAnlysingContext, ContractDocument
from contract_patterns import ContractPatternFactory
from documents import TextMap
from legal_docs import LegalDocument
from ml_tools import SemanticTag
from structures import ContractTags

# UTIL

In [0]:
parser_lib_version='1.0.8'
!wget https://github.com/nemoware/document-parser/releases/download/$parser_lib_version/document-parser-$parser_lib_version-distribution.zip
!unzip document-parser-$parser_lib_version-distribution.zip


## Export Contract Pattern Factory

In [0]:
import pickle
from contract_patterns import ContractPatternFactory

contract_pattern_factory = ContractPatternFactory(GLOBALS__['elmo_embedder'])
contract_pattern_factory.embedder=None

with open('contract_pattern_factory.pickle', 'wb') as handle:
  pickle.dump(contract_pattern_factory, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# from contract_parser import ContractDocument
from integration.word_document_parser import WordDocParser, join_paragraphs
from legal_docs import LegalDocument, Paragraph
from ml_tools import SemanticTag
import pickle, os

# '/content/gdrive/My Drive/GazpromOil/Contracts/3 этап по правилам/Договор_2_.doc'
# FN = 'Договор _2_.docx'
# FILENAME=f'/content/gdrive/My Drive/GazpromOil/Contracts/3 этап по правилам/{FN}'
FN='Протокол_СД_ 3.docx'
FILENAME=f'/content/{FN}'
os.environ ['documentparser']=f'/content/document-parser-{parser_lib_version}'
 
res = WordDocParser().read_doc(FILENAME)

#------------------------
cd = join_paragraphs(res, FILENAME)
#------------------------
 
cd.embedd_tokens(GLOBALS__['elmo_embedder'])
print('embedded ok', cd.embeddings.shape)

with open(f'{FN}.pickle', 'wb') as handle:
  pickle.dump(cd, handle, protocol=pickle.HIGHEST_PROTOCOL)

 

In [0]:
print(' '.join( cd.tokens_cc))

In [0]:
with open(f'{FN}.pickle', 'rb') as handle:
  b = pickle.load(handle)
  print('embedded ok', b.embeddings.shape)

#Tests

In [0]:
  def test_find_contract_subject_region_in_doc_head(doc, ctx):
    section = doc.subdoc_slice(slice(0, 1500))
    denominator = 0.7
 
    # ----------------------------------------
    result = ctx.find_contract_subject_regions(section, denominator)
    # ---------------------

    self.print_semantic_tag(result, doc.tokens_map)
    # assert  '1.1 Благотворитель оплачивает следующий счет, выставленный на Благополучателя:'== doc.tokens_map.text_range(result.span).strip())



test_find_contract_subject_region_in_doc_head()
  